# Project Cars4you (Group 5): Explorative Data Analysis

Cars 4 You is an online car resale company that sells cars from multiple different brands.  
Their main goal is to expedite the evaluation process by creating a predictive model capable of evaluating the price of a car based on the user’s input without needing the car to be taken to a mechanic.

| **Attribute**       | **Description** |
|----------------------|-----------------|
| **carID**            | An attribute that contains an identifier for each car. |
| **Brand**            | The car’s main brand (e.g. Ford, Toyota) |
| **model**            | The car model |
| **year**             | The year of Registration of the Car |
| **mileage**          | The total reported distance travelled by the car (in miles) |
| **tax**              | The amount of road tax (in £) that, in 2020, was applicable to the car in question. |
| **fuelType**         | Type of Fuel used by the car (Diesel, Petrol, Hybrid, Electric) |
| **mpg**              | Average Miles per Gallon (fuel efficiency) |
| **engineSize**       | Size of engine in liters (cubic decimeters) |
| **paintQuality%**    | The mechanic’s assessment of the cars’ overall paint quality and hull integrity (filled by the mechanic during evaluation). |
| **previousOwners**   | Number of previous registered owners of the vehicle |
| **hasDamage**        | Boolean marker filled by the seller at the time of registration stating whether the car is damaged or not |
| **price (TARGET)**            | The car’s price when purchased by Cars 4 You (in £) |

TASK I (3 Points): Descriptive Statistics, Inconsistency Check, Visual Data Explorance, Extraction of Relevant Insights, Multivariate Relationships  => Explain

# 1. Import & load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import joblib
import seaborn as sns
from data_cleaning import clean_car_dataframe
from itertools import combinations
from scipy.stats import chi2_contingency

In [ ]:
df_cars_train = pd.read_csv("train.csv")
df_cars_test = pd.read_csv("test.csv")

# 2. Basic Overview

In [ ]:
# Overview of structure and data types
df_cars_train.info()

# Strip whitespace and lowercase before duplicate check
df_cars_train = df_cars_train.replace(r'^\s*$', np.nan, regex=True)

# Check for duplicates
print(f"\nDuplicate rows: {df_cars_train.duplicated().sum()}") # 0

# Check for duplicate carID
print(f"\nDuplicate carID: {df_cars_train['carID'].duplicated().sum()}") # 0
print(f"Null values in column carID: {df_cars_train['carID'].isnull().sum()}") # 0
print(f"All carID entries are unique: {df_cars_train['carID'].nunique()}/{len(df_cars_train)}")


# Findings:
#   - missing values in every column except carID and price
#   - year, mpg, previousOwners, hasDamage as float seems weird
#   - no duplicates in all columns

In [ ]:
display(df_cars_train.describe().T)
display(df_cars_train.describe(include="object").T)

### First Findings (extensive description of each attribute in Univariate Section):
# Numerical:
#   - carID is unique identifier
#   - years range from 1970 to 2024.12 (a float so maybe some have month info?) with most of the cars being between 2016 and 2019 (max value of 2024 doesnt make sense because database is from 2020)
#   - price ranges from 450 to 159.999 with mean ~17.000, median ~14.500 and std ~10.000 -> right skewed distribution (some cars with high prices)
#   - mileage ranges from a negative value (probably wrong data entry) to 323.000 with mean ~23.000, median 17300 and std ~22.000 -> right skewed distribution (some cars with very high mileage) and high variance
#   - tax ranges from a negative value (probably wrong data entry) to 580 with mean ~120, median 145 and std ~65 -> left skewed distribution (some cars with low tax)
#   - mpg ranges from a negative value (probably wrong data entry) to 470,8 with mean ~55, median 54,3 and std ~16 -> not skewed
#   - engineSize ranges from a negative value (probably wrong data entry) to 6,6 with mean ~1,6 median 1.6 and std ~0,5 -> not skewed
#   - paintQuality% ranges from 1,6 to 125,6 (should be 0-100% -> probably wrong data entry) with mean ~65, median 28,7 and std ~12 -> not skewed
#   - previousOwners ranges from a negative value (probably wrong data entry) to 6,3 (shouldnt be float values -> probably wrong data entry) with mean ~2 median 2 and std ~1,5 -> not skewed but high variance
#   - hasDamage is only 0 values -> probably wrong data entries
# Categorical:
#   - 72 unique brands (though multpiple wrong data entries result in less unique brands (see EDA below))
#   - 735 unique models
#   - transmission with 40 unique values -> probably wrong data entries because it should be somewhat limited to manual and automatic
#   - fuelType with 34 unique values -> probably wrong data entries because it should be limited to petrol, diesel, electric, hybrid

In [ ]:
# print exact unique values of df_cars_train
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())

# 3. Separation of numerical and categorical features

In [ ]:
# Standardize categorical variables (strip whitespace, lowercase)
cat_cols = [col for col in df_cars_train.columns if df_cars_train[col].dtype == 'object']
print("Categorical columns:", cat_cols)
for col in cat_cols:
    df_cars_train[col] = df_cars_train[col].str.strip().str.lower()

In [ ]:
num_cols = [col for col in df_cars_train.columns if df_cars_train[col].dtype in ['int64', 'float64']]
num_cols.remove('carID') # remove carID because its an identifier
num_cols.remove('hasDamage') # remove hasDamage because its only 0 or nan values and even when imputing nan to 1 it has no correlatin with price
df_cars_train['hasDamage'].value_counts(dropna=False)
damaged_to_1 = df_cars_train['hasDamage'].fillna(1).astype('Int64')
damaged_to_1.value_counts(dropna=False)
hasDamage_price_corr = damaged_to_1.corr(df_cars_train['price']).round(4)

print(f"hasDamage with filled NaN to 1 correlation with price ({hasDamage_price_corr}) is ~0 so its not worth it to make the assumption that NaN means damaged => The feature will be ignored.")

print("Numerical columns:", num_cols)

In [ ]:
neither_categorical_numerical = [col for col in df_cars_train.columns if col not in cat_cols + num_cols]
neither_categorical_numerical.remove('carID')
neither_categorical_numerical.remove('hasDamage')
print("Neither categorical nor numerical columns:", neither_categorical_numerical)

# 4. Univariate Analysis

In [ ]:
# Visualization settings
sns.set(style="whitegrid")
hist_color = '#1f77b4'   # dark blue
box_color = '#ff7f0e'    # warm orange

## 4.1 Numerical

In [ ]:
def plot_numerical_univariate_distributions(df, cols):
    # figure: 2 features per row (4 plots per row = hist + boxplot per feature)
    n_features = len(cols)
    n_rows = int(np.ceil(n_features / 2))
    fig, axes = plt.subplots(n_rows, 4, figsize=(16, n_rows * 4))
    axes = axes.flatten()
    
    for i, col in enumerate(cols):
        # histogram
        sns.histplot(df[col], bins=25, color=hist_color, kde=False, ax=axes[i*2])
        axes[i*2].set_xlabel('')
        axes[i*2].set_ylabel('')
        axes[i*2].set_title(f'{col}', fontsize=11, pad=12)

        if df[col].dtype in ['int64', 'float64', 'Int64', 'Float64']:
            # add mean and median lines
            mean_val = df[col].mean()
            median_val = df[col].median()
            axes[i*2].axvline(mean_val, color='red', linestyle='--', label='Mean')
            axes[i*2].axvline(median_val, color='green', linestyle='-', label='Median')
            axes[i*2].legend()

            # boxplot (vertical)
            sns.boxplot(y=df[col], ax=axes[i*2 + 1], color=box_color)
            axes[i*2 + 1].set_xlabel('')
            axes[i*2 + 1].set_ylabel('')
            axes[i*2 + 1].set_title(f'{col}', fontsize=11, pad=12)

    # hide any unused axes
    for j in range(i*2 + 2, len(axes)):
        fig.delaxes(axes[j])

    plt.suptitle('univariate distributions of numerical features', fontsize=14, y=1.01)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_numerical_univariate_distributions(df_cars_train, num_cols)

In [ ]:
# Analyze the gap in paintQuality
print(df_cars_train['paintQuality%'].unique())
display(df_cars_train[df_cars_train['paintQuality%'] < 4].sort_values('paintQuality%'))
display(df_cars_train[df_cars_train['paintQuality%'] >= 4].sort_values('paintQuality%'))
df_cars_train[df_cars_train['paintQuality%'].between(4, 29)]
print("==> Remove paintQuality values below 5 and above 100 as outliers in data cleaning step. These values are not just outliers but also stored as float values which is inconsistent with the other values.")

## 4.2 Categorical

In [ ]:
# plot_univariate_distributions(cat_cols)
def plot_categorical_distributions(df, cols):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    axes = axes.flatten()
    top_n = 20

    for ax, col in zip(axes, cols):
        vc = df[col].value_counts().nlargest(top_n)
        sns.barplot(x=vc.index, y=vc.values, color=hist_color, ax=ax)
        ax.set_title(f'{col}')
        ax.set_xlabel('')
        ax.set_ylabel('count')
        ax.tick_params(axis='x', rotation=90)
        ax.set_xticklabels(ax.get_xticklabels(), ha='right')

    # hide any unused axes (in case cat_cols length < axes)
    for i in range(len(cat_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
print("Top 20 most frequent values per categorical column:")
plot_categorical_distributions(df_cars_train, cat_cols)

In [ ]:
df_cars_train['Brand'].value_counts()

In [ ]:
df_cars_train['model'].value_counts()

In [ ]:
df_cars_train['transmission'].value_counts()

In [ ]:
df_cars_train['fuelType'].value_counts()

## 4.3 Findings after Descriptive Statistics and Inconsistency Check

<div class="alert alert-info">
    <b>Attention:</b> The target variable (price) is right-skewed -> requires log-transform for linear models.
</div>

<div class="alert alert-warning">
    <b>Warning:</b> Some <b>numerical features</b> (mileage, tax, mpg, engineSize, previousOwner) contain negative entries, which is not possible. Data cleaning is necessary to ensure a more accurate analysis.
</div>

<div class="alert alert-warning">
    <b>Warning:</b> All <b>categorical features</b> (brand, model, transmission and fuelType) contain numerous spelling inconsistencies. Data cleaning is necessary to ensure accurate and interpretable results.
</div>

**Numerical:**

price (Target variable):  
- range **£450–£159,999**, mean ≈ **£16.9k**, median ≈ **£14.7k** → right-skewed (log-transform useful before building a linear regression model)  
- typical cars priced **£10k–£21k**, few luxury outliers inflate mean  
- consistent integer values, no missing or obvious anomalies  
- strong dependence expected on **mileage**, **year**, **engine size**, and **brand**

carID:  
- sequential numeric identifier, ranges from **0–75,972**  
- no duplicates expected, used only as index/key  

year:  
- values range from **1970–2024**, mean ≈ **2017**  
- years after **2020** are unrealistic (future registration) → invalid entries  
- Two cars in year **1970** are outliers with a big gap to the following cars (starting from 1996)
- decimals in year (e.g. **2023.367**) → data corruption, round to nearest int  
- older outliers before **2000** rare, likely classic or miscoded entries  
- can derive **age = 2020 - year** for modeling  

mileage:  
- range **–58,540 → 323,000**, mean ≈ **23k** → negatives invalid  
- missing values around **1.5k**  
- strong right skew, some extreme outliers >**250k miles**  
- negative or zero values should be filtered or replaced with abs()  

tax:  
- range **–91 → 580**, mean ≈ **120** → invalid negatives present  
- normal values cluster around **125–145**  
- decimals and small negatives appear due to calculation/entry errors  
- likely strong right skew → a few cars taxed over **500**  
- needs capping and replacement for negatives  

mpg:  
- range **–43 → 470**, mean ≈ **55** → negative and extreme outliers exist  
- typical real range **30–70 mpg**, but some values like **470** unrealistic  
- invalid entries indicate unit mix-up or input noise  
- expected inverse relation with **engine size** and **price**  

engineSize:  
- range **–0.1 → 6.6L**, mean ≈ **1.66L**, std ≈ **0.57**  
- several decimals and negative/zero values → invalid  
- expected valid range **0.6–6.0L**  
- most cars between **1.2–2.0L** → compact to mid-size engines  
- positive correlation with **price** and **tax**  

paintQuality%:  
- range **1.6 → 125.6%**, mean ≈ **64.6%**  
- values above **100%** unrealistic → scaling error  
- some extremely low values (**≈1–3%**) indicate outliers or noise  
- most cars between **50–80%** → average paint quality  
- **No values between 4 and 29** → Remove values < 5 and > 100 because they are not only outliers but also stored as float values which is inconsistent with the other values suggesting wrong data entry.

previousOwners:  
- range **–2.3 → 6.26**, mean ≈ **2**  
- negative values invalid → likely placeholder or encoding issue  
- most between **0–3** → typical secondhand ownership distribution  
- outliers >**6** likely data entry errors or mis-scaling  
- likely negative correlation with **price**  

hasDamage:  
- only values are **0 and NaN** → no variation  
- unclear if NaN means damaged → convert to int and verify meaning  
- likely nonfunctional feature → **drop**  


**Categorical:**

brand:  
- **72 unique brands** with severe spelling and capitalization inconsistencies (**Ford**, **ford**, **FOR**, **ord**, **For**, etc.)  
- contains partial or truncated names (**w**, **MW**, **Ope**, **Mercede**) → heavy cleaning required  
- **Ford dominates (~15k entries)**, followed by **BMW**, **VW**, **Mercedes**, **Toyota**, **Audi**, **Skoda**, **Hyundai**  
- strong class imbalance → use one hot encoding, frequency encoding or median price per brand later  

model:  
- **735 unique entries** with inconsistent formatting, spacing, capitalization, and partial strings  
- duplicates of same model under variations (e.g. *“focus”*, *“ FOCUS”*, *“ Focu”*, *“Focus”*)  
- many small typos, truncated or malformed entries (*“Focu”*, *“EcoSpor”*, *“Gol”*, *“Yeti Outdoo”*, etc.)  
- heavy normalization needed → strip whitespace, lowercase, and fix common truncations  
- dominated by popular models like **Ford Focus**, **VW Golf**, **Vauxhall Astra/Corsa**, **Skoda Octavia**  

transmission:  
- **40 distinct entries**, mostly spelling variants of *manual*, *automatic*, *semi-auto*  
- common corruptions: *manua*, *anual*, *semi-aut*, *utomatic*, *nknow*, etc.  
- some leading/trailing spaces (*' manual '*, *' Manual '*)  
- categories should be reduced to clean labels: **manual**, **automatic**, **semi-auto**, **unknown**

fuelType:
- lots of spelling errors
- should be limited to petrol, diesel, hybrid and electric

</div>


# 5. Univariate after data cleaning

In [ ]:
print(f"NaN entries because of mileage < 0: {len(df_cars_train[df_cars_train['mileage'] < 0])}")
print(f"NaN entries because of tax < 0: {len(df_cars_train[df_cars_train['tax'] < 0])}. Of those already NaN because of mileage: {len(df_cars_train[(df_cars_train['tax'] < 0) & (df_cars_train['mileage'] < 0)])}")
print(f"NaN entries because of mpg < 5 and > 150: {len(df_cars_train[df_cars_train['mpg'] < 5])} and {len(df_cars_train[df_cars_train['mpg'] > 150])}. Of those already NaN because of mileage: {len(df_cars_train[(df_cars_train['mpg'] < 5) & (df_cars_train['mileage'] < 0)]) + len(df_cars_train[(df_cars_train['mpg'] > 150) & (df_cars_train['mileage'] < 0)])}. Because of tax: {len(df_cars_train[(df_cars_train['mpg'] < 5) & (df_cars_train['tax'] < 0)]) + len(df_cars_train[(df_cars_train['mpg'] > 150) & (df_cars_train['tax'] < 0)])}")
print(f"NaN entries because of engineSize < 0 and > 9 : {len(df_cars_train[df_cars_train['engineSize'] < 0])} and {len(df_cars_train[df_cars_train['engineSize'] > 9])}. Of those already NaN because of mileage: {len(df_cars_train[(df_cars_train['engineSize'] < 0) & (df_cars_train['mileage'] < 0)])}. Because of tax: {len(df_cars_train[(df_cars_train['engineSize'] < 0) & (df_cars_train['tax'] < 0)])}. Because of mpg: {len(df_cars_train[(df_cars_train['engineSize'] < 0) & (df_cars_train['mpg'] < 5)]) + len(df_cars_train[(df_cars_train['engineSize'] < 0) & (df_cars_train['mpg'] > 150)])}")
print(f"Nan entries because of paintQuality < 0 and > 100: {len(df_cars_train[(df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100)])}. Of those already NaN because of mileage: {len(df_cars_train[((df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100)) & (df_cars_train['mileage'] < 0)])}. Because of tax: {len(df_cars_train[((df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100)) & (df_cars_train['tax'] < 0)])}. Because of mpg: {len(df_cars_train[((df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100)) & (df_cars_train['mpg'] < 5 | (df_cars_train['mpg'] > 150))])}. Because of engineSize: {len(df_cars_train[((df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100)) & (df_cars_train['engineSize'] < 0 | (df_cars_train['engineSize'] > 9))])}")
print(f"Nan entries because of previousOwners < 0 : {len(df_cars_train[df_cars_train['previousOwners'] < 0])}. Of those already NaN because of mileage: {len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['mileage'] < 0)])}. Because of tax: {len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['tax'] < 0)])}. Because of mpg: {len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['mpg'] < 5)]) + len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['mpg'] > 150)])}. Because of engineSize: {len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['engineSize'] < 0)]) + len(df_cars_train[(df_cars_train['previousOwners'] < 0) & (df_cars_train['engineSize'] > 9)])}. Because of paintQuality: {len(df_cars_train[(df_cars_train['previousOwners'] < 0) & ((df_cars_train['paintQuality%'] < 0) | (df_cars_train['paintQuality%'] > 100))])}")

print("\n ==> Barely any overlaps between NaN entries of different columns.")

In [ ]:
print(f"Number of rows without missing values: {df_cars_train.dropna(inplace=False).shape[0]}")
print(f"Length of dataset before cleaning: {len(df_cars_train)}")
df_cars_train_cleaned = clean_car_dataframe(df_cars_train)
print(f"\nNumber of rows without missing values after cleaning: {df_cars_train_cleaned.dropna(inplace=False).shape[0]}")
print(f"Length of dataset after cleaning: {len(df_cars_train_cleaned)}")

## 5.1 Numerical

In [ ]:
num_cols.remove('paintQuality%')
num_cols.append('paintQuality')
plot_numerical_univariate_distributions(df_cars_train_cleaned, num_cols)

## 5.2 Categorical

In [ ]:
plot_categorical_distributions(df_cars_train_cleaned, cat_cols)

In [ ]:
df_cars_train_cleaned['Brand'].value_counts(dropna=False)

In [ ]:
df_cars_train_cleaned['model'].value_counts(dropna=False)

In [ ]:
df_cars_train_cleaned['transmission'].value_counts(dropna=False)

In [ ]:
df_cars_train_cleaned['fuelType'].value_counts(dropna=False)

## 5.3 New Findings after cleaning

**Numerical**:

paintQuality:
- ranges from 30-100 now without the (probably invalid) outliers
- roughly equally distributed

mpg:
- After the cleaning, the distribution looks very **symmetric** with a roughly mean=median=54

Other numerical features:
- Except for that, no notable new findings, just cleaner distributions without the invalid negative values (mileage, tax, mpg, engineSize, previousOwner) and unrealistic percentage values (paintQuality).


**Categorical:**

brand:
- **Only 9 unique brands** left after cleaning the strings
- from 3400 (Hyundai) to ~16.000 (Ford) entries
- **108 NaNs** that will be imputed in preprocessing

model:
- **Only 114 unique models** left after cleaning the strings
- **4 cars** with **over 3500 entries** dominate the dataset (focus, c class, fiesta, golf)
- **5121 NaNs** that will be imputed in preprocessing

transmission:
- **Most** of the cars are **Manual** (41615/75973)
- **Semi-Auto and Automatic** are roughly **equally distributed** (16867 and 15205)
- Some entries are Unknown (735) or marked as Other (5) -> has to be dealt with in preprocessing
- **1546 NaNs** that will be imputed in preprocessing

fuelType:
- Nearly all cars are **Petrol or Diesel** (41181 and 30885)
- Some Hybrids (2225) and **barely no Electrics (4)** -> has to be paid attention when feature of car in test set is electric
- Some marked as Other (167) -> has to be dealt with in preprocessing
- **1511 NaNs** that will be imputed in preprocessing

# 6. Bivariate Analysis

Bivariate analysis is performed on the cleaned data to ensure a reliable analysis that is not hindered by invalid values.

## 6.1 Numerical-Numerical Associations

### 6.1.1 Pairplot

Pairplots to visualize the relationships before computing metrics like correlation to quantify them.

In [ ]:
g = sns.pairplot(df_cars_train_cleaned[num_cols], diag_kind='kde', corner=True) # NAs are included because they will be imputed and therefore should be part of the distribution
g.figure.suptitle('Numeric Features — Pairwise Relationships', y=1.02)
plt.show()

<div class="alert alert-info">
    <b>Attention:</b> Heteroscedastic relationships between target (price) and predictors (years, mileage) violate the assumption of linear models → Linear regression models are probably not be the best choice for this dataset (log-transform necessary).
</div>

**Findings** (more details in the Findings Correlation section)

Price (Target variable):
- **Very strong positive and heteroscedastic relationship with years**: The most expensive cars are the newest cars with the spread for new cars being higher than for old cars → Cheap new cars exist while there is no expensive very old car in the data
- **Strong Negative and heteroscedastic relationship with mileage**: As mileage increases, the price decreases. At low mileage, there is a huge range of possible prices. At high mileage, all cars have a low price, and the spread is very small. → Some older cars have been driven long distances, while others have relatively low mileage despite their age
- **Negative and heteroscedastic relationship with mpg**: However, low-mpg cars have a very wide range of prices while high-mpg cars are almost all in the low-price bracket.
- **Strong positive and heteroscedastic relationship with engineSize**: As engineSize increases, price tends to increase. No very expensive cars with very small engine size.
- **Relationship seems to be negative with previousOwners**: Cars with many previous owners are found only in the lowest price bracket.
- **No clear relationship with paintQuality**: Values appear to be spread randomly across all price points → Variable will likely be a poor predictor of price

Relationships between independent variables will be analyzed explicitly in the Correlation Section.

### 6.1.2 Correlation

**Why we decided to use Spearman correlation:**

- the numeric variables (price, mileage, tax, engineSize) are **non-normally distributed and contain outliers**, causing linear measures like pearson to distort correlation strength.  
- **spearman** evaluates **monotonic** relationships based on rank order rather than exact linearity, making it **robust to skewness and outliers**.  
- this allows us to correctly capture the direction and magnitude of real-world trends, such as price decreasing with mileage, even if not perfectly linear.  
- **pearson correlation** is plotted for linear modeling checks, but **spearman is more appropriate for initial EDA**.

In [ ]:
# pearson and spearman correlation
corr_pearson = df_cars_train_cleaned[num_cols].corr(method='pearson', numeric_only=True).round(2)
corr_spearman = df_cars_train_cleaned[num_cols].corr(method='spearman', numeric_only=True).round(2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))
mask = np.triu(np.ones_like(corr_spearman, dtype=bool))

sns.heatmap(corr_pearson, mask=mask, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, fmt='.2f',
            linewidths=0.5, ax=axes[0], cbar=False)
axes[0].set_title('pearson correlation (linear)')

sns.heatmap(corr_spearman, mask=mask, annot=True, cmap='coolwarm',vmin=-1, vmax=1, center=0, fmt='.2f',
            linewidths=0.5, ax=axes[1], cbar_kws={'label': 'correlation strength'})
axes[1].set_title('spearman correlation (monotonic)')

plt.suptitle('comparison of pearson vs spearman correlation', fontsize=14, y=1.05)
plt.tight_layout()
plt.show()

**Findings based on spearman correlation:**

Price (target variable):
- price shows **strong positive correlation with year (0.60)** → newer cars are priced higher  
- price is **positively correlated with engineSize (0.57)** → larger engines increase car value  
- price is **moderately negatively correlated with mileage (–0.52)** → more driven cars lose value  
- price correlates **negatively with mpg (–0.39)** → efficient cars are typically smaller and cheaper  
- tax has a **moderate positive correlation (0.32)** with price → more expensive cars often have higher taxes  

Correlations between independent variables (potential multicollinearity, strong correlation indicates overlapping information about the dependent variable):
- year and mileage have a **very strong negative correlation (–0.79)** → newer cars have lower mileage 
- year has a **moderate positive correlation with tax (0.32)** -> newer cars have higher tax
- year has a **moderate negative correlation with mpg (–0.32)** -> newer cars are more efficient
- mileage is **moderately negatively correlated with tax (-0.26)** -> more driven cars have lower tax
- mileage is **moderately correlated with mpg (0.33)** -> more driven cars are less efficient
- tax and mpg are **strongly negatively correlated (–0.56)** → efficient cars usually taxed less
- mpg and engineSize **correlate negatively (–0.21)** → larger engines are less fuel-efficient  
- **paintQuality and previousOwners** show **near-zero correlations** with all other variables → low predictive relevance  

=> Overall, **price mainly depends on year, engineSize, mileage, and mpg**, which align with intuitive market behavior

## 6.2 Numerical-Categorical Associations

Use boxplots to see whether numerical features vary depending on the category.

In [ ]:
sns.set(style="whitegrid")
sns.set_palette("vlag")

# One figure (grid) per numerical variable in relation to all categorical variables
def plot_categorical_boxplots(num, cat_cols):
    n = len(cat_cols)
    rows = math.ceil(n / 3)
    cols = min(3, n)

    fig, axes = plt.subplots(rows, cols, figsize=(16, 5 * rows))
    axes = axes.flatten()

    for ax, cat in zip(axes, cat_cols):
        sns.boxplot(
            data=df_cars_train_cleaned,
            x=cat,
            y=num,
            ax=ax
        )
        ax.set_title(f"{num} by {cat}")
        ax.tick_params(axis="x", rotation=90)

    # hide any unused axes
    for ax in axes[n:]:
        ax.set_visible(False)

    fig.suptitle(f"Distribution of {num} across categorical features", fontsize=16, y=1.02)
    plt.tight_layout()
    plt.show()

cat_cols_for_bivariate = cat_cols.copy()
cat_cols_for_bivariate.remove('model')  # too many unique values
for num in num_cols:
    plot_categorical_boxplots(num, cat_cols_for_bivariate)

**Findings**

Price (Target variable):
- Audi, BMW and Mercedes are the most expensive brands
- There are no Manual cars with very high prices
- The most expensive cars are petrol cars

Years:
- "Other" transmissions has no old data entry -> could indicate new process for registering transmission
- "Eletric" fuelType has no old data entries -> could indicate new developments in the market

Mileage:
- Highest mileage cars are Diesel cars -> Diesel cars seem to have the longest lifetime

Tax:
- For Hybrid and Electric vehicles are no high taxes recorded -> Either because of lack of enough samples or because they are taxed less than other fuel types

Mpg:
- BMW has multiple outliers with a high mpg -> potentially big cars with low fuel efficiency
- Hybrid cars have higher mpg than petrol or diesel cars -> probably because of electric motor assistance

engineSize:
- varies a lot for BMW and Mercedes -> potentially some cars with high-performance engines and some with standard engines
- the biggest engines are petrol cars

Distribution of previous owners and paintQuality across categorical features don't provide any insights

## 6.3 Categorical-Categorical Associations

Analysis of categorical associations is not immediately important because we don't see the relationship with the numeric target variable (price). However, it is important to understand how the other features relate to each other.

Stacked bar chart to show how one categorical variable is distributed within another → useful to spot **group differences**, **patterns**, and **imbalances** in the data.  
Cramér’s V complements the visuals with a metric.

### 6.3.1 Stacked bar charts

In [ ]:
df_cars_train_cleaned_stacked_bars = df_cars_train_cleaned.copy()
print("Categorical columns for bivariate analysis:", cat_cols_for_bivariate)
# Helper to compute %
def stacked_percent(df, g1, g2):
    tmp = df[[g1, g2]].dropna()
    share = (
        tmp.groupby([g1, g2]).size()
        .groupby(level=0).apply(lambda x: x / x.sum() * 100)
        .unstack(fill_value=0)
    )
    return share

pairings = list(combinations(cat_cols_for_bivariate , 2))

# Plot grid
n = len(pairings)
cols = 3
rows = -(-n // cols)  # ceiling division

fig, axes = plt.subplots(rows, cols, figsize=(20, rows*4))
axes = axes.ravel()

sns.set_style("whitegrid")
sns.set_palette("vlag")

for ax, (cat1, cat2) in zip(axes, pairings):
    dist = stacked_percent(df_cars_train_cleaned_stacked_bars, cat1, cat2)
    dist.plot(kind="bar", stacked=True, ax=ax, alpha=0.85)

    ax.set_title(f"{cat2} by {cat1}", fontsize=10)
    ax.set_xlabel(cat1)
    ax.set_ylabel("Percent")
    ax.tick_params(axis="x", rotation=90)
    ax.legend(title=cat2, bbox_to_anchor=(1.05, 1), fontsize=8)

# Hide unused axes if any
for ax in axes[len(pairings):]:
    ax.set_visible(False)

plt.tight_layout()
plt.show()


**Findings**
- Cars from Ford and Opel are predominantly manual and petrol-fueled -> likely older and more affordable models
- Manual transmission cars are primarily petrol-fueled

### 6.3.2 Cramér's V

**Cramér’s V** is a statistical measure that quantifies how strongly two **categorical variables** are associated: The higher the value, the higher the association.

- It is based on the **Chi-square (χ²) statistic**, which compares the **observed frequencies** in a contingency table to the **expected frequencies** (if the variables were independent).  
- Cramér’s V normalizes this χ² value to produce a result **between 0 and 1**, making it easy to interpret.

\[
V = \sqrt{\frac{\chi^2}{n \times (k - 1)}}
\]

Where:
- **χ²** = Chi-square statistic from the contingency table  
- **n** = total number of observations  
- **k** = smaller of (number of rows, number of columns)

**Key Points**
- Symmetric: same result whether you swap the variables.  
- Works only for **categorical–categorical** relationships.  
- Does **not** tell the *direction* of association (only strength).  

In [ ]:
# Cramér's V for categorical features
def cramers_v(x, y):
    tbl = pd.crosstab(x, y)
    if tbl.shape[0] < 2 or tbl.shape[1] < 2:
        return np.nan
    chi2 = chi2_contingency(tbl, correction=False)[0]
    n = tbl.values.sum()
    r, k = tbl.shape
    phi2 = chi2 / n
    # bias correction
    phi2_corr = max(0, phi2 - (k - 1)*(r - 1)/(n - 1))
    r_corr = r - (r - 1)**2/(n - 1)
    k_corr = k - (k - 1)**2/(n - 1)
    denom = max((k_corr - 1), (r_corr - 1))
    return np.sqrt(phi2_corr / denom) if denom > 0 else np.nan

if len(cat_cols) > 1:
    m = pd.DataFrame(index=cat_cols, columns=cat_cols, dtype=float)

    for a in cat_cols:
        for b in cat_cols:
            if a == b:
                m.loc[a, b] = 1.0
            else:
                m.loc[a, b] = cramers_v(df_cars_train_cleaned[a], df_cars_train_cleaned[b])

    plt.figure(figsize=(min(0.7 * len(cat_cols) + 5, 14),
                        min(0.7 * len(cat_cols) + 5, 14)))

    sns.heatmap(
        m.astype(float),
        annot=True,
        fmt=".2f",
        cmap="vlag",
        vmin=0, vmax=1,
        linewidths=.5,
        annot_kws={"size": 10},
        alpha=0.65,
        cbar_kws={"shrink": 0.85}
    )

    plt.title("Cramér's V — categorical features")
    plt.xticks(rotation=45, ha="right")
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
else:
    print("Not enough categorical features for Cramér's V.")


**Findings**
- The strongest but still only weak association is between **brand and model** (0.27) which makes sense but the low value suggests that there are many models for each brand, so knowing the brand doesn't perfectly predict the model.

The other associations are too weak and not directly related to our target variable (price) so not worth mentioning.

# 7. Multivariate Analysis

Some 3-dimensional analysis regarding specific relationships with the target variable (price).

In [ ]:
# TODO ~J